In [1]:
import json
import logging
import os
from typing import NoReturn

from pydantic import ValidationError

from core import ExcelHandler, NetworkSimulation, PipSimInput

logger = logging.getLogger("Hydraulics")

In [2]:
from wave_hydraulics import *

In [3]:
config = load_config("inputs.json")
ns = NetworkSimulation(
    config.FOLDER_DIRECTORY, config.MODEL_FILENAME, config.EXCEL_FILE
)
ns.open_model()

INFO:Hydraulics:Input file loaded successfully.
INFO:NetworkSimulation:------------Network Simulation Object Created----------------
Model Path: c:\Users\IDM252577\Desktop\Python Projects\PIPSIM\BAB LP.pips

INFO:manta.server.manager:Starting PIPESIM server on thread ID: 19236
INFO:manta.server.manager:Python toolkit license is checked out successfully
INFO:manta.server.manager:Waiting for PIPESIM server to start.


In [4]:
ns.get_all_values()

INFO:NetworkSimulation:Getting all values.....


In [5]:
ns.model_inputs

,component,type
0,common,None
1,Unit,FlowLine
2,,None
3,J 162,Junction
4,J 163,Junction
...,...,...
3932,FL 994:FL 994,None
3933,FL 994:Default,None
3934,FL 993:FL 993,None
3935,FL 993:Default,None


In [7]:
ns_values = ns.values

In [9]:
ns.model_inputs["type"].value_counts()

type
FlowLine    985
Junction    491
Well        487
Name: count, dtype: int64

In [13]:
ExcelHandler.write_excel(
    ns.model_inputs,
    sheet_name="rough",
    workbook=ns.excel_filename,
    clear_sheet=False,
    range="E1",
)

In [10]:
ns.excel_filename

WindowsPath('Water Injection Profile for BAB LP.xlsx')

In [14]:
ns.model.open_ui()

In [6]:
import pandas as pd

file = r"C:\Users\IDM252577\Downloads\data.xlsx"

df = pd.read_excel(file, sheet_name="Sheet1", header=3)
dt = df.to_dict()
print(dt)

{'BranchName': {0: 'J 174 J 162', 1: 'J 162_WIC - 24', 2: 'J 281 WIC - 24', 3: 'J 280 281', 4: 'J 277 3 280', 5: 'J 275 3 277', 6: 'J 325 3 275', 7: 'J 274 3 325', 8: 'J 250 3 274', 9: 'J 253 3 250', 10: '3 273 3 253', 11: 'J 252_3 273', 12: '3 272 J 252', 13: 'J 75 J 272', 14: 'WIC - 23_J 75', 15: 'J 71_WIC - 23', 16: 'J 80 3 71', 17: 'LP - 2101 J 80', 18: 'J 23 LP - 2101', 19: 'J 22 3 23', 20: 'J 21 22', 21: 'J 20 J 21', 22: 'J 19 3 20', 23: '317_319', 24: 'J 16 3 17', 25: '16 3_37 נ', 26: 'J 14 3 37', 27: 'J 12 3 14', 28: 'J 93 3 12', 29: 'J 93 93', 30: 'J7_J9', 31: 'J6_37', 32: 'J5_36', 33: 'J 345 3 5', 34: 'J 3 3 345', 35: 'J 446 3 3', 36: 'J 445 FE - 2100-01 3 446', 37: 'J 445 FE - 2100-04_3 447', 38: 'J 447 3 91', 39: 'J 91 94', 40: 'J 94 J 100', 41: 'J 100 J 108', 42: 'J 108 ) 111', 43: 'J 1113 116', 44: 'J 116 3 125', 45: 'J 125 J 132', 46: 'J 132_J 134', 47: 'J 134 J 136', 48: 'J 136_J 138', 49: 'J 138 J 139', 50: 'J 139 3 141', 51: 'J 141 142', 52: 'J 142_3 145', 53: 'J 145 

In [7]:
from pprint import pprint

pprint(dt)

{'BranchName': {0: 'J 174 J 162',
                1: 'J 162_WIC - 24',
                2: 'J 281 WIC - 24',
                3: 'J 280 281',
                4: 'J 277 3 280',
                5: 'J 275 3 277',
                6: 'J 325 3 275',
                7: 'J 274 3 325',
                8: 'J 250 3 274',
                9: 'J 253 3 250',
                10: '3 273 3 253',
                11: 'J 252_3 273',
                12: '3 272 J 252',
                13: 'J 75 J 272',
                14: 'WIC - 23_J 75',
                15: 'J 71_WIC - 23',
                16: 'J 80 3 71',
                17: 'LP - 2101 J 80',
                18: 'J 23 LP - 2101',
                19: 'J 22 3 23',
                20: 'J 21 22',
                21: 'J 20 J 21',
                22: 'J 19 3 20',
                23: '317_319',
                24: 'J 16 3 17',
                25: '16 3_37 נ',
                26: 'J 14 3 37',
                27: 'J 12 3 14',
                28: 'J 93 3 12',
        

In [8]:
df

,BranchName,ElevationIn,ElevationOut,Direction,DeltaElevation,TotalDeltaElevation,JunctionName
0,J 174 J 162,0.000000,4.000195,forward,-4.000195,-4.000195,J 174
1,J 162_WIC - 24,0.000000,0.999744,forward,-0.999744,-4.999939,J 162
2,J 281 WIC - 24,3.999890,0.000000,reversed,3.999890,-1.000049,WIC - 24
3,J 280 281,3.000146,0.000000,reversed,3.000146,2.000098,J 281
4,J 277 3 280,0.000000,0.000000,reversed,0.000000,2.000098,J 280
...,...,...,...,...,...,...,...
62,J 168 3 169,0.000000,-1.999793,forward,1.999793,-5.700674,J 168
63,J 169 J 170,0.000000,-5.000244,forward,5.000244,-0.700430,J 169
64,J 170 J 172,0.000000,2.000098,forward,-2.000098,-2.700528,J 170
65,J 172_3 173,0.000000,-2.000098,forward,2.000098,-0.700430,J 172


In [5]:
import xlwings as xw


def get_last_row(workbook: str, sheet_name: str):
    with xw.App(visible=False) as app:
        wb = xw.Book(workbook)
        ws = wb.sheets(sheet_name)
        last_row = ws.range("B1").end("down").row
        return last_row

In [7]:
get_last_row("Node Results.xlsx", "Node Summary")

8